### join air quality and grid weather data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# air quality data path
aq_data_path = "./data/air_quality_fillthree.csv"
gw_data_path = "./data/grid_weather_1701_1804.csv"
geo_info_path = "./data/geo_info_new.csv"

In [13]:
aq_new_data_path = './data/air_quality_full.csv'

In [14]:
aq_df_new = pd.read_csv(aq_new_data_path,index_col=0)

In [7]:
aq_df = pd.read_csv(aq_data_path,index_col=0)
gw_df = pd.read_csv(gw_data_path,index_col=None)
geo_info = pd.read_csv(geo_info_path,index_col=None)

In [8]:
def join_aq_gw(aq,gw,geo_info,start_time,end_time):
    aq = aq[(aq.time >= start_time) & (aq.time <= end_time)]
    gw = gw[(gw.time >= start_time) & (gw.time <= end_time)]
    
    # find unique gw_name in geo_info dataframe
    near_gw_stations = geo_info.nearest_gw.unique()
    # filter these gw_station from gw_df
    near_gw_df = gw[gw.stationId.isin(near_gw_stations)]
    
    # first join air quality data and geo_info to add aq_station's nearest gw_station
    geo_info = geo_info.rename(columns={"aq_name":"stationId"})
    aq_gw_foreign = geo_info[["stationId","nearest_gw"]]
    intermediate = aq.join(aq_gw_foreign.set_index("stationId"),on="stationId")
    
    nearest_gw_df = gw.rename(columns={"stationId":"nearest_gw"})
    
    # then join intermediate with grid_data
    joined = intermediate.join(nearest_gw_df.set_index(["nearest_gw","time"]), on=["nearest_gw","time"])
    
    return joined

In [9]:
joined = join_aq_gw(aq_df, gw_df, geo_info,"2017-01-01 00:00:00","2018-04-30 23:00:00")

In [10]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406910 entries, 0 to 406909
Data columns (total 16 columns):
stationId         406910 non-null object
time              406910 non-null object
PM2.5             366417 non-null float64
PM10              308543 non-null float64
O3                367745 non-null float64
year              406910 non-null int64
month             406910 non-null int64
day               406910 non-null int64
weekday           406910 non-null int64
hour              406910 non-null int64
nearest_gw        406910 non-null object
humidity          402920 non-null float64
pressure          402920 non-null float64
temperature       402920 non-null float64
wind_direction    402920 non-null float64
wind_speed        402920 non-null float64
dtypes: float64(8), int64(5), object(3)
memory usage: 52.8+ MB


In [12]:
joined.to_csv('./data/total_data_without_aqfull.csv')

In [16]:
joined_new = join_aq_gw(aq_df_new, gw_df, geo_info,"2017-01-01 00:00:00","2018-04-30 23:00:00")
joined_new.to_csv('./data/total_data.csv')